In [ ]:
%run helpers/DataFrameLoader.ipynb

In [ ]:
h54_loader = DataFrameLoader.from_file('data/054.xlsx')
h99_loader = DataFrameLoader.from_file('data/099.xlsx')

In [ ]:
h54_loader.reset()
h54_loader.add_features('power', sheet_name='smartMeter')
h54_pow_df = h54_loader.df
h99_loader.reset()
h99_loader.add_features('power', sheet_name='smartMeter')
h99_pow_df = h99_loader.df

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

print('power')
plt.subplots()
plt.plot(h54_pow_df)
plt.plot(h99_pow_df)
plt.show()
print('054 - skew', h54_pow_df['smartMeter_power'].skew())
print('099 - skew', h99_pow_df['smartMeter_power'].skew())
print('054 - kurt', h54_pow_df['smartMeter_power'].kurtosis())
print('099 - kurt', h99_pow_df['smartMeter_power'].kurtosis())

In [ ]:
h54_loader.reset()
h54_loader.add_features('co2', sheet_name='co2sensor')
h54_co2_df = h54_loader.df
h99_loader.reset()
h99_loader.add_features('co2', sheet_name='co2sensor')
h99_co2_df = h99_loader.df

In [ ]:
plt.subplots()
plt.plot(h54_co2_df)
plt.plot(h99_co2_df)
plt.show()
print('054 - skew', h54_co2_df.skew()['co2sensor_co2'])
print('099 - skew', h99_co2_df.skew()['co2sensor_co2'])
print('054 - kurt', h54_co2_df.kurtosis()['co2sensor_co2'])
print('099 - kurt', h99_co2_df.kurtosis()['co2sensor_co2'])

In [ ]:
h54_loader.reset()
h54_loader.add_features('flow_temp', sheet_name='alklimaHeatPump')
h54_temp_df = h54_loader.df
h99_loader.reset()
h99_loader.add_features('flow_temp', sheet_name='alklimaHeatPump')
h99_temp_df = h99_loader.df

In [ ]:
plt.subplots()
plt.plot(h54_temp_df)
plt.plot(h99_temp_df)
plt.show()
print('054 - skew', h54_temp_df.skew()['alklimaHeatPump_flow_temp'])
print('099 - skew', h99_temp_df.skew()['alklimaHeatPump_flow_temp'])
print('054 - kurt', h54_temp_df.kurtosis()['alklimaHeatPump_flow_temp'])
print('099 - kurt', h99_temp_df.kurtosis()['alklimaHeatPump_flow_temp'])

In [ ]:
import pandas as pd

targets = [
    {
        "conf": "knmi",
        "from": "344 Rotterdam.csv",
        "to": "260 De Bilt.csv",
        "target": ("0", "Temperature")
    },
    {
        "conf": "knmi",
        "from": "344 Rotterdam.csv",
        "to": "260 De Bilt.csv",
        "target": ("0", "Relative atmospheric humidity")
    },
    {
        "conf": "knmi",
        "from": "344 Rotterdam.csv",
        "to": "260 De Bilt.csv",
        "target": ("0", "Global Radiation")
    },
    {
        "conf": "factory zero",
        "from": "054.xlsx",
        "to": "099.xlsx",
        "target": ("alklimaHeatPump", "flow_temp")
    },
    {
        "conf": "factory zero",
        "from": "054.xlsx",
        "to": "099.xlsx",
        "target": ("alklimaHeatPump", "op_mode")
    },
    {
        "conf": "factory zero",
        "from": "054.xlsx",
        "to": "099.xlsx",
        "target": ("smartMeter", "power")
    },
    {
        "conf": "factory zero",
        "from": "054.xlsx",
        "to": "099.xlsx",
        "target": ("co2sensor", "co2")
    }
]

dfloaders = {}


def get_dfloader(conf, file):
    global dfloaders
    if file in dfloaders.keys():
        dfloaders[file].reset()
        return dfloaders[file]
    %run helpers/load_data.ipynb
    config_select.value = conf
    file_select.value = file
    dfloaders[file] = dfloader
    return dfloader

knmi_df = None
f0_df = None

for t in targets:
    res = None
    if t["conf"] == "knmi":
        if knmi_df is None:
            knmi_df = pd.DataFrame(columns=[
                f'Kurtosis for {t["from"].split(".")[0]}',
                f'Kurtosis for {t["to"].split(".")[0]}',
                'Kurtosis absolute difference',
                f'Skewness for {t["from"].split(".")[0]}',
                f'Skewness for {t["to"].split(".")[0]}',
                'Skewness absolute difference'
            ])
        res = knmi_df
    elif t["conf"] == "factory zero":
        if f0_df is None:
            f0_df = pd.DataFrame(columns=[
                f'Kurtosis for {t["from"].split(".")[0]}',
                f'Kurtosis for {t["to"].split(".")[0]}',
                'Kurtosis absolute difference',
                f'Skewness for {t["from"].split(".")[0]}',
                f'Skewness for {t["to"].split(".")[0]}',
                'Skewness absolute difference'
            ])
        res = f0_df

    from_dfl = get_dfloader(t["conf"], t["from"])
    to_dfl = get_dfloader(t["conf"], t["to"])

    from_dfl.add_targets(t['target'][1], sheet_name=t['target'][0])
    to_dfl.add_targets(t['target'][1], sheet_name=t['target'][0])

    from_kurt = from_dfl.df.kurtosis()[from_dfl.targets[0]]
    to_kurt = to_dfl.df.kurtosis()[to_dfl.targets[0]]
    res.loc[(t["target"][1], res.columns[0])] = from_kurt
    res.loc[(t["target"][1], res.columns[1])] = to_kurt
    res.loc[(t["target"][1], res.columns[2])] = abs(from_kurt - to_kurt)

    from_skew = from_dfl.df.skew()[from_dfl.targets[0]]
    to_skew = to_dfl.df.skew()[to_dfl.targets[0]]
    res.loc[(t["target"][1], res.columns[3])] = from_skew
    res.loc[(t["target"][1], res.columns[4])] = to_skew
    res.loc[(t["target"][1], res.columns[5])] = abs(from_skew - to_skew)

In [7]:
display(knmi_df)
display(f0_df)

,Kurtosis for 344 Rotterdam,Kurtosis for 260 De Bilt,Kurtosis absolute difference,Skewness for 344 Rotterdam,Skewness for 260 De Bilt,Skewness absolute difference
Temperature,-0.223448,-0.265535,0.042087,0.211647,0.240832,0.029186
Relative atmospheric humidity,0.269973,-0.00999,0.279964,-0.889676,-0.847846,0.04183
Global Radiation,2.227166,2.316581,0.089415,1.764216,1.7817,0.017484


,Kurtosis for 054,Kurtosis for 099,Kurtosis absolute difference,Skewness for 054,Skewness for 099,Skewness absolute difference
flow_temp,3.19863,2.752413,0.446216,1.667278,1.416585,0.250693
op_mode,5.778331,3.582762,2.195569,2.417315,1.603526,0.813789
power,1.73615,1.429675,0.306475,-1.008938,-1.084015,0.075077
co2,7.650241,2.235878,5.414363,1.905655,1.334289,0.571365
